## anansi_.marketdata.klines

In [1]:
from anansi_.marketdata.klines import klines_getter
from anansi_.marketdata.models import Market

In [2]:
my_market = Market(broker_name="Binance", ticker_symbol="BTCUSDT")


my_klines_getter = klines_getter(my_market)#, time_frame="6h")

In [3]:
my_klines = my_klines_getter.newest(100)

In [ ]:
my_klines.apply_indicator.trend.price_from_kline()

In [ ]:
class SetupSMA:
    price_metrics = "hlc3"
    number_samples = 20

my_setup = SetupSMA()

my_klines.apply_indicator.trend.simple_moving_average(my_setup)

In [4]:
my_klines

,Open_time,Open,High,Low,Close,Volume
1,2020-11-25 14:36:00,19183.54,19187.00,19163.40,19170.42,55.786067
2,2020-11-25 14:37:00,19170.41,19188.88,19170.41,19186.25,55.992697
3,2020-11-25 14:38:00,19185.75,19200.00,19181.07,19200.00,71.436440
4,2020-11-25 14:39:00,19200.00,19212.10,19199.09,19205.00,70.187918
5,2020-11-25 14:40:00,19205.00,19220.00,19200.00,19211.52,86.483567
...,...,...,...,...,...,...
96,2020-11-25 16:11:00,19166.54,19183.84,19164.03,19171.25,64.247077
97,2020-11-25 16:12:00,19171.25,19183.38,19169.99,19178.13,70.333961
98,2020-11-25 16:13:00,19178.13,19178.13,19135.46,19143.74,65.933844
99,2020-11-25 16:14:00,19143.74,19147.00,19100.00,19110.54,77.213724


In [ ]:
my_market.broker_name

## anansi_.marketdata.brokers

###  Exemplo com a Binance

In [ ]:
from anansi_.marketdata.brokers import Binance

my_broker = Binance()

### Alguns atributos da exchange/broker

In [ ]:
my_broker.base_endpoint, my_broker.DateTimeFmt, my_broker.DateTimeUnit

### Verificando horário do servidor com broker.server_time()

In [ ]:
import pendulum # Só para gerar o formato 'human readable', para exemplo

server_time = dict(server_time_timestamp=my_broker.server_time(),
                   server_time_human_readable=(
                       pendulum.from_timestamp(my_broker.server_time()).to_datetime_string()
                   )
                  )

server_time

### Verificando se o limite de requests foi atingido

In [ ]:
my_broker.was_request_limit_reached()

### Finalmente, solicitando algumas klines

In [ ]:
my_klines = my_broker.get_klines(ticker_symbol="BTCUSDT", time_frame="2h")

my_klines

### Função get_broker

É possível repetir todos os passos acima, desde que haja suporte para o broker (passado para a função 'get_broker' pelo parâmetro 'broker_name') instanciando um broker como mostrado abaixo (novamente tomando a binance como exemplo):

In [ ]:
from anansi_.marketdata.brokers import get_broker

my_broker = get_broker(broker_name="binance")

Testando com as klines; esteja livre para repetir os passos anteriores :)

In [ ]:
my_klines = my_broker.get_klines(ticker_symbol="BTCUSDT", time_frame="2h")

my_klines

In [ ]:
type(my_klines)